# Předzpracování datasetu CIFAR10

Tento notebook slouží k předzpracování datasetu CIFAR10. Pro dataset jsou vytvořeny augmentované záznamy a předpočítány logity.
Nejprve jsou načteny všechny potřebné knihovny včetně vlastní sbírky objektů a funkcí.

In [1]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from transformers import AutoModelForImageClassification
from torch.utils.data import ConcatDataset, DataLoader
import torch
import os

import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Ověření, že GPU je k dispozici a balíček torch je správně nakonfigurován.

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [ ]:
base.reset_seed(42)

Získání základních transformací pro spuštění inference nad učitelským modelem. Základní transformace primárně obrázky zvětšují na rozměr 224x224 a normalizují barevné kanály. 

Transformace sloužící k augmentaci poté tyto kroky rozšiřují o rotaci a převracení os, vždy s určitou pravděpodobností. 

Transformace pochází z balíčku torchvision.

In [ ]:
transform = base.base_transforms()
augment_transform = base.aug_transforms()

Získání již natrénovaného učitele z HuggingFace.

In [6]:
model = AutoModelForImageClassification.from_pretrained(
    "aaraki/vit-base-patch16-224-in21k-finetuned-cifar10",
    num_labels=10,
)

model.to(device)
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/cifar10/teacher.pth")

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [7]:
model.eval()

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [ ]:
dataset_part = base.get_dataset_part()

Načtení již staženého datasetu CIFAR10. Nejprve se otevírá přímo stažený soubor pro přímý přístup k datům. Následně se obsah souboru načte taktéž do objektu, který dataset představuje a umožňuje s ním pohodlně pracovat. Objekt představující dataset aplikuje požadované transformace a přesouvá dataset na GPU (pokud je k dispozici).

Následně se skrze dataloader dataset předává učitelskému modelu pro provedení inference a získání predikcí, které jsou k původnímu datasetu uloženy. V tomto případě jsou tyto kroky provedeny pro testovací a validační část datasetu. Validační část datasetu je v tomto případě tvořena 10 000 záznamů z trénovací částí (trénovací část je rozdělena do 5 souborů po 10 000 záznamech, pracujeme s posledním), jelikož dataset tuto část ve výchozím stavu neobsahuje.   

In [9]:
testing = base.unpickle(f"{os.path.expanduser('~')}/data/10/cifar-10-batches-py/test_batch")
test_data = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", train=False, transform=transform, device=device)
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=False)

logits_test = base.generate_logits(test_dataloader, model)
testing[b"logits"] = logits_test
base.pickle_up(f"{os.path.expanduser('~')}/data/10-logits/cifar-10-batches-py/test", testing)




evaluating = base.unpickle(f"{os.path.expanduser('~')}/data/10/cifar-10-batches-py/data_batch_5")
eval_data = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", train=True, batch=5, transform=transform, device=device)
eval_dataloader = DataLoader(eval_data, batch_size=128, shuffle=False)

logits_eval = base.generate_logits(eval_dataloader, model)
evaluating[b"logits"] = logits_eval
base.pickle_up(f"{os.path.expanduser('~')}/data/10-logits/cifar-10-batches-py/eval", evaluating)

Generating logits for given dataset:   0%|          | 0/79 [00:00<?, ?it/s]

Generating logits for given dataset:   0%|          | 0/79 [00:00<?, ?it/s]

Vytvořené upravené podkladové soubory jsou načteny a je nad nimi spočtena správnost. Tedy do jaké míry jsou učitelské predikce spolehlivé.

In [10]:
test = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TEST, transform=transform)
eval = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.EVAL, transform=transform)

print(base.check_acc(test, "Accuracy for test dataset is:"))
print(base.check_acc(eval, "Accuracy for eval dataset is:"))

Calculating accuracy based on the saved logits:   0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy for test dataset is: 0.9508


Calculating accuracy based on the saved logits:   0%|          | 0/10000 [00:00<?, ?it/s]

Accuracy for eval dataset is: 0.9583


Předpočítání logitů pro trénovací část probíhá stejným způsobem jako v předchozím případě. Postupně jsou načítány jednotlivé soubory obsahující trénovací data. Na data jsou aplikovány normální i augmentační transformace, přičemž pro obě varianty jsou získány predikce učitele, které jsou ukládány do výchozího souboru. 

In [11]:
base.reset_seed(42)
for index in range(1,5):
    data = base.unpickle(f"{os.path.expanduser('~')}/data/10/cifar-10-batches-py/data_batch_{index}")

    train = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", batch=index, train=True, transform=transform, device=device)
    train_augmented = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", batch=index, train=True, transform=augment_transform, device=device)
    
    train_dataloader = DataLoader(train, batch_size=64, shuffle=False)
    train_dataloader_augmented = DataLoader(train_augmented, batch_size=64, shuffle=False)

    logits_arr = base.generate_logits(train_dataloader, model)
    logits_arr_aug = base.generate_logits(train_dataloader_augmented, model) 

    data[b"logits"] = logits_arr
    data[b"logits_aug"] = logits_arr_aug
    base.pickle_up(f"{os.path.expanduser('~')}/data/10-logits/cifar-10-batches-py/train_batch_{index}",data)

Upravené datasety jsou opět pro ověření načteny a je nad nimi spočítáná správnost učitelských predikcí. 

Nejprve je ověřena správnost nad trénovací částí se základními transformacemi, následovaná částí pouze s augmentacemi. Jako poslední je ověřena kombinace obou částí.

In [12]:
train_aug = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=augment_transform)
train = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=transform)
train_combo = ConcatDataset([train, train_aug])

Vzhledem k velmi slabé správnosti nad augmentovanou částí datasetu (kombinace zvětšení a následného otáčení a přetáčení) bylo přistoupeno k filtraci záznamů. V případě destilace je klíčové, aby se na učitele bylo možné spolehnout, což ani kombinace augmentovaného a výchozího datasetu neumožňuje. 

In [13]:
print(base.check_acc(train, "Accuracy for train dataset is:"))
print(base.check_acc(train_aug, "Accuracy for augmeted train dataset is:"))
print(base.check_acc(train_combo, "Accuracy for combined dataset is:"))

Calculating accuracy based on the saved logits:   0%|          | 0/40000 [00:00<?, ?it/s]

Accuracy for train dataset is: 0.954925


Calculating accuracy based on the saved logits:   0%|          | 0/40000 [00:00<?, ?it/s]

Accuracy for augmeted train dataset is: 0.686


Calculating accuracy based on the saved logits:   0%|          | 0/80000 [00:00<?, ?it/s]

Accuracy for combined dataset is: 0.8204625


Filtrace záznamů probíhá na základě rozdílných predikcí učitele nad výchozím a augmentovaném datasetu. 
- Data jsou seřazena stejně.
- Na základě spočtených logitů se určí predikce učitele pro výchozí dataset.
- Na základě spočtených logitů se určí predikce učitele pro augmentovaný dataset.
- Pokud se predikce liší, záznam je vymazán.

Tímto způsobem nedochází k umělému navýšení výkonu učitele, vyfiltrovány totiž nejsou všechny chybné predikce. 

In [14]:
train_aug = base.remove_diff_pred_class(train, train_aug, pytorch_dataset=True)
train_combo = ConcatDataset([train, train_aug])

Removing entries from augmented dataset that are different from the base one - based on saved logits:   0%|   …

Tímto přístupem přijdeme o relativně velkou část záznamů, nicméně správnost nad augmentovaným datasetem je nyní porovnatelná s výchozím. Student se tedy na predikce může více spoléhat. 

In [15]:
print(len(train_aug))

28176


In [16]:
print(base.check_acc(train_aug, "Accuracy for filtered augmented dataset is:"))
print(base.check_acc(train_combo, "Accuracy for combined dataset is:"))

Calculating accuracy based on the saved logits:   0%|          | 0/28176 [00:00<?, ?it/s]

Accuracy for filtered augmented dataset is: 0.9614565587734242


Calculating accuracy based on the saved logits:   0%|          | 0/68176 [00:00<?, ?it/s]

Accuracy for combined dataset is: 0.9576243839474302


Následně jsou již pouze získány informace o učitelském modelu (velikost, rychlost inference a další výkonnostní metriky nad datasetem).

In [17]:
base.count_parameters(model)

model size: 327.325MB.
Total Trainable Params: 85806346.


,Modules,Parameters
0,vit.embeddings.cls_token,768
1,vit.embeddings.position_embeddings,151296
2,vit.embeddings.patch_embeddings.projection.weight,589824
3,vit.embeddings.patch_embeddings.projection.bias,768
4,vit.encoder.layer.0.attention.attention.query....,589824
...,...,...
195,vit.encoder.layer.11.layernorm_after.bias,768
196,vit.layernorm.weight,768
197,vit.layernorm.bias,768
198,classifier.weight,7680


In [18]:
train_part_cpu = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", batch=1, train=True, transform=transform, device="cpu")
cpu_data_loader = DataLoader(train_part_cpu, batch_size=1, shuffle=False)
cpu_benchmark = base.BenchMarkRunner(model, cpu_data_loader, "cpu", 1000)

print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  220.89 ms
  1 measurement, 1000 runs , 4 threads


In [19]:
train_part_gpu = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", batch=1, train=True, transform=transform, device="cuda")
gpu_data_loader = DataLoader(train_part_gpu, batch_size=1, shuffle=False)
gpu_benchmark = base.BenchMarkRunner(model, gpu_data_loader, "cuda", 1000)


print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  12.05 ms
  1 measurement, 1000 runs , 4 threads


In [4]:
transform = base.base_transforms()
dataset_part = base.get_dataset_part()
test = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TEST, transform=transform)

In [ ]:
test_data_preds = []
test_data_labels = []

for index, val in enumerate(test):
    test_data_preds.append(torch.topk(val["logits"], k=1).indices.numpy()[0])
    test_data_labels.append(val["labels"])

In [34]:
f1 = f1_score(test_data_labels, test_data_preds, average="macro")
acc = accuracy_score(test_data_labels, test_data_preds)
precision = precision_score(test_data_labels, test_data_preds, average="macro")
recall = recall_score(test_data_labels, test_data_preds, average="macro")

print(f"F1 score: {f1}")
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

F1 score: 0.9508764449007329
Accuracy: 0.9508
Precision: 0.9529362112257411
Recall: 0.9507999999999999
